In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pytest
import tempfile
import json
from pathlib import Path
from core.common.theme_manager import ThemeManager, ConfigManager

## Test ConfigManager

In [ ]:
def test_config_manager_init():
    """Test ConfigManager initialization"""
    with tempfile.TemporaryDirectory() as tmpdir:
        config = ConfigManager(config_dir=tmpdir)
        
        assert config.config_dir == Path(tmpdir)
        assert config.config_dir.exists()
        print("✅ ConfigManager init test passed")

test_config_manager_init()

In [ ]:
def test_config_set_get():
    """Test setting and getting config values"""
    with tempfile.TemporaryDirectory() as tmpdir:
        config = ConfigManager(config_dir=tmpdir)
        
        # Set values
        config.set('theme', 'dark')
        config.set('notifications.enabled', True)
        
        # Get values
        assert config.get('theme') == 'dark'
        assert config.get('notifications.enabled') == True
        assert config.get('nonexistent', 'default') == 'default'
        print("✅ Config set/get test passed")

test_config_set_get()

In [ ]:
def test_config_persistence():
    """Test config persistence across instances"""
    with tempfile.TemporaryDirectory() as tmpdir:
        # First instance
        config1 = ConfigManager(config_dir=tmpdir)
        config1.set('test_key', 'test_value')
        config1.save()
        
        # Second instance (should load saved config)
        config2 = ConfigManager(config_dir=tmpdir)
        config2.load()
        
        assert config2.get('test_key') == 'test_value'
        print("✅ Config persistence test passed")

test_config_persistence()

## Test ThemeManager

In [ ]:
def test_theme_manager_init():
    """Test ThemeManager initialization"""
    with tempfile.TemporaryDirectory() as tmpdir:
        theme_mgr = ThemeManager(config_dir=tmpdir)
        
        assert hasattr(theme_mgr, 'themes')
        assert 'dark' in theme_mgr.themes
        assert 'light' in theme_mgr.themes
        assert theme_mgr.current_theme in ['dark', 'light']
        print(f"✅ ThemeManager init test passed (current: {theme_mgr.current_theme})")

test_theme_manager_init()

In [ ]:
def test_theme_definitions():
    """Test theme definitions"""
    with tempfile.TemporaryDirectory() as tmpdir:
        theme_mgr = ThemeManager(config_dir=tmpdir)
        
        # Check dark theme
        dark = theme_mgr.themes['dark']
        assert 'bg' in dark
        assert 'fg' in dark
        assert 'accent' in dark
        
        # Check light theme
        light = theme_mgr.themes['light']
        assert 'bg' in light
        assert 'fg' in light
        assert 'accent' in light
        
        print("✅ Theme definitions test passed")

test_theme_definitions()

In [ ]:
def test_toggle_theme():
    """Test theme toggling"""
    try:
        import tkinter as tk
        from tkinter import ttk
        
        with tempfile.TemporaryDirectory() as tmpdir:
            theme_mgr = ThemeManager(config_dir=tmpdir)
            
            # Create temporary root
            root = tk.Tk()
            root.withdraw()  # Hide window
            
            initial_theme = theme_mgr.current_theme
            theme_mgr.toggle_theme(root)
            new_theme = theme_mgr.current_theme
            
            # Should be different
            assert initial_theme != new_theme
            
            # Toggle back
            theme_mgr.toggle_theme(root)
            assert theme_mgr.current_theme == initial_theme
            
            root.destroy()
            print(f"✅ Toggle theme test passed: {initial_theme} ⇄ {new_theme}")
            
    except ImportError:
        print("⚠️ Toggle theme test skipped (tkinter not available)")

test_toggle_theme()

In [ ]:
def test_theme_persistence():
    """Test theme preference persistence"""
    with tempfile.TemporaryDirectory() as tmpdir:
        # First instance - set to dark
        theme_mgr1 = ThemeManager(config_dir=tmpdir)
        theme_mgr1.current_theme = 'dark'
        theme_mgr1.config.set('theme', 'dark')
        theme_mgr1.config.save()
        
        # Second instance - should load dark theme
        theme_mgr2 = ThemeManager(config_dir=tmpdir)
        
        assert theme_mgr2.current_theme == 'dark'
        print("✅ Theme persistence test passed")

test_theme_persistence()

## Test Window Geometry

In [ ]:
def test_window_geometry():
    """Test window geometry save/restore"""
    try:
        import tkinter as tk
        
        with tempfile.TemporaryDirectory() as tmpdir:
            config = ConfigManager(config_dir=tmpdir)
            
            # Create window
            root = tk.Tk()
            root.withdraw()
            root.geometry('800x600+100+50')
            
            # Save geometry
            config.save_window_geometry(root)
            
            # Check saved values
            assert config.get('window.width') == 800
            assert config.get('window.height') == 600
            
            root.destroy()
            print("✅ Window geometry test passed")
            
    except ImportError:
        print("⚠️ Window geometry test skipped (tkinter not available)")

test_window_geometry()

## Run All Tests

In [ ]:
# Run all tests
pytest.main(['-v', '--tb=short', __file__])